In [1]:
import pandas as pd
import numpy as np

In [2]:
sales_data = pd.read_pickle('sales_full_enh.pkl')

In [120]:
sales_simple = sales_data[['shop_id', 'item_id', 'date_block_num', 'month', 'city', 'item_cnt_month', 'avg_item_price', 'category_type']]
sales_simple.item_cnt_month = sales_simple.item_cnt_month.fillna(0)
sales_simple['item_cnt_month'] = sales_simple.item_cnt_month.astype('float32')
sales_simple['avg_item_price'] = sales_simple.avg_item_price.astype('float32')
sales_simple.ffill(inplace=True)
sales_simple.fillna(0, inplace=True)

C:\Users\Adrian.Maciej.KYNETEC\AppData\Local\Continuum\anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Adrian.Maciej.KYNETEC\AppData\Local\Continuum\anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [121]:
sales_simple['history'] = list(zip(
                                    sales_simple.month,
                                    sales_simple.city, 
                                    sales_simple.category_type, 
                                    sales_simple.item_cnt_month, 
                                    sales_simple.avg_item_price))

C:\Users\Adrian.Maciej.KYNETEC\AppData\Local\Continuum\anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
sales_simple.merge()

In [133]:
next_month = sales_data[['predict_item_cnt_month']]
sales_merged = sales_simple.merge(next_month, left_index=True, right_index=True)
sales_merged.drop(['month', 'city','item_cnt_month', 'avg_item_price', 'category_type'], axis=1, inplace=True)
sales_merged.columns = ['shop_id', 'item_id', 'date_block_num', 'history', 'prediction']
sales_merged.fillna(0, inplace=True)

In [106]:
sales_grouped = sales_merged.sort_values('date_block_num').groupby(['shop_id', 'item_id'])['history'].apply(list)
sales_grouped = sales_grouped.reset_index()
res = pd.merge(sales_grouped[['history']], sales_merged[['prediction']], left_index=True, right_index=True)

In [153]:
train_set = sales_merged[(sales_merged.date_block_num > 30-12) & (sales_merged.date_block_num < 32)]
validation_set = sales_merged[(sales_merged.date_block_num > 32-12) & (sales_merged.date_block_num < 33)]
test_set = sales_merged[sales_merged.date_block_num > 33-12]

In [141]:
list(sales_merged.history[0:12]), sales_merged.prediction[12]

([(0, 0, 7, 0.0, 0.0),
  (1, 0, 7, 0.0, 0.0),
  (2, 0, 7, 1.0, 359.0),
  (3, 0, 7, 0.0, 359.0),
  (4, 0, 7, 0.0, 359.0),
  (5, 0, 7, 1.0, 399.0),
  (6, 0, 7, 0.0, 399.0),
  (7, 0, 7, 0.0, 399.0),
  (8, 0, 7, 0.0, 399.0),
  (9, 0, 7, 0.0, 399.0),
  (10, 0, 7, 0.0, 399.0),
  (11, 0, 7, 0.0, 399.0)],
 0.0)